In [ ]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
import numpy as np 

In [ ]:
def plot_y(y_pred, y_true):
    y_pred = y_pred.reshape(-1)
    y_true = y_true.reshape(-1)
    order = np.argsort(y_pred)
    plt.plot(y_true[order], 'o ', label=u'réel')
    plt.plot(y_pred[order], 'P ', label=u'prédiction')
    plt.legend()
    plt.xlabel(u'échantillons')
    plt.ylabel('valeur de y')

# Optimization d'un MLP

Dans cet exercice nous allons réaliser l'optimization d'un perceptron à une couche caché. 
On se servira de pytorch et sa fonctionalité autograd pour apliquer la descente du gradient.


## Les données : Predire la progression du diabetes

On continuera à travailler avec les données sur la progression du diabetes. Le code ci dessus charge les données dans des variables, les normalize et les divide en "train" et "test", exactement comme dans le notebook antérieur.

In [ ]:
from sklearn.datasets import load_diabetes

data = load_diabetes()

#print(data.DESCR)

X, y = data.data, data.target

# import pandas as pd
# pd.DataFrame(X[:10,:], columns=data.feature_names)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

y_train, y_test = y_train.reshape([-1,1]), y_test.reshape([-1,1])

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## MLP en numpy

Ici je reprends le code pour le MLP qui à été fait dans le notebook antérieur.

In [ ]:
M = X_train.shape[0]  # nombre d'échantillons
D = X_train.shape[1]  # nombre de features
S = y_train.shape[1]  # nombre de sorties

In [ ]:
def parameters(D, S, N=10):
    # N le nombre de neurones de la couche caché
    rng = np.random.RandomState(0)
    
    # changez les tailles des matrices ici pour leurs valeurs correctes
    # tailleW = (0,0)
    # tailleb = (0,0)
    # tailleO = (0,0)
    tailleW = (D,N)
    tailleb = (N,1)
    tailleO = (N,S)
  
    W = rng.rand(*tailleW)
    b = np.zeros(tailleb)
    
    O = rng.rand(*tailleO)
    
    return W, b, O

In [ ]:
def MLP(X, W, b, O):
    M, D = X.shape  # nombre d'échantillons, nombre de features
    N = W.shape[1]  # N le nombre de neurones de la couche caché
    
    # écrivez le calcul de H
    # H = np.zeros(1) # changez ici pour avoir la bonne equation de H
    # A = np.tanh(H)  
    H = np.dot(X,W) + b.T # changez ici pour avoir la bonne equation de H
    A = np.tanh(H)  

    # écrivez le calul de Y
    # Y = np.zeros(1) # changez ici pour avoir la bonne equation de Y
    Y = np.dot(A,O) # changez ici pour avoir la bonne equation de Y
    
    # Ici quelques verifications sur la taille des matrices pour vous aider
    try:
        assert(H.shape == (M,N))
    except AssertionError:
        print("Taille de H semble erronée:",H.shape, ", ça devrait être", (M,N))    

    try:
        assert(Y.shape == (M,S))
    except AssertionError:
        print("Taille de Y semble erronée:",Y.shape, ", ça devrait être", (M,S))
    
    return Y, A

In [ ]:
def cout(Y_pred, Y_true):
    M = Y_true.shape[0]
    # completez le code avec l'expression pour la fonction de cout J
    # J = 0 # changez ici pour avoir la bonne equation de J
    J = (1/2)*np.mean((Y_pred - Y_true)**2)
    return J

In [ ]:
W, b, O = parameters(D, S, N=5)

In [ ]:
Y_pred, A = MLP(X_train, W, b, O)

In [ ]:
J = cout(Y_pred, y_train)
J

## Le gradient de J
Comme pour la regression lineaire, il nous faudra calculer les gradients de J envers les poids et biais du reseau. 
$$grad_O J = A^T (Y-\hat{Y})$$
$$grad_b = \mathbb{1}_M^T \left[(Y-\hat{Y}) O^T \odot (1-A^2)\right] $$
$$grad_W = X^T  \left[(Y-\hat{Y}) O^T \odot (1-A^2)\right]  $$
$$A^2:=\{(a_i^k)^2\}$$

In [ ]:
def gradients(X, Y_pred, Y_true, W, b, O, A):
    dH = 1-A**2
    terme_YOA = np.matmul(Y_pred-Y_true, O.T)* dH
    gradJb = np.mean(terme_YOA, axis=0, keepdims=True).T
    assert(b.shape == gradJb.shape)
    
    gradJW = # completez
    assert(W.shape == gradJW.shape)
    
    gradJO = # completez
    assert(O.shape == gradJO.shape)


    return gradJW, gradJb, gradJO

In [ ]:
# Exemple d'utilisation
Y_pred, A = MLP(X_train, W, b, O)
gradients(X_train, Y_pred, y_train, W, b, O, A)

### Exercice : pas du gradient

Maintenant vous savez comment calculer les gradients et voir leurs valeurs. A vous de remplir la fonction cidessus avec les mises à jour pour la descente du gradient.
Vous allez metre à jour W, b et O.

In [ ]:
def gradient_step(learning_rate, X, Y_pred, Y_true, W, b, O, A):
    lr = learning_rate
    
    # Calcul des gradients
    gradJW, gradJb, gradJO = gradients(X, Y_pred, Y_true, W, b, O, A)

    # Metez à jour W, b et O 
    # dans la direction oposé du gradient
    # completez
    W -= lr * gradJW




### Exercice : boucle d'optimization
Tout est prêt pour l'entrainement de notre réseau. Il faut maintenant créer la boucle d'optimization qui realise la descente de gradient pour W, b, et O. Suivez les indications et completez le code ci-dessous.

In [ ]:
W, b, O = parameters(D, S, N=5) # N est le nombre de neurones de la couche caché
max_iterations = 1000
learning_rate = 1e-5
cost_curve = []
for i in range(max_iterations):
    #Completez le code ci dessous (numeros 1 a 3)
    # 1) calculez les predictions avec le réseau (forward pass)
    Y_pred, A =
    
    # 2) Calculez la fonction de cout
    J = 
    
    # Sauvegarder J pour plot
    cost_curve.append(J)
    
    # 3) Calculez les gradients et metez a jour W, b et o avec la fonction gradient_step
    gradient_step

    

Si tout se passe bien vous allez voir ci-dessus l'evolution de la valeur de la focntion de coût au long des iterations.

In [ ]:
plt.plot(cost_curve)
plt.title("courbe d'apprentissage")
plt.xlabel('iterations')

In [ ]:
plot_y(Y_pred, y_train)

## Évaluation du modèle

Finalement, on poura tester la qualité de notre modéle sur notre ensemble de teste ( qui n'a pas été utilisé pour l'apprentissage).

In [ ]:
Y_pred, A = MLP(X_test, W, b, O)
J = cout(Y_pred,y_test)
print("cout sur l'ensemble de test", J)
plot_y(Y_pred, y_test)

### Exercice : essayez d'augmenter le nombre d'iterations, de neurones ou changer la learning rate pour voir si le modèle peut mieux faire !

## Corrigés
### Gradients
``` python
def gradients(X, Y_pred, Y_true, W, b, O, A):
    dH = 1-A**2
    terme_YOA = np.matmul(Y_pred-Y_true, O.T)* dH
    gradJb = np.mean(terme_YOA, axis=0, keepdims=True).T
    assert(b.shape == gradJb.shape)
    
    # gradJW = # completez
    gradJW = np.matmul(X.T, terme_YOA)
    assert(W.shape == gradJW.shape)
    
    # gradJO = # completez
    gradJO = np.matmul(A.T, (Y_pred-Y_true))
    assert(O.shape == gradJO.shape)

    return gradJW, gradJb, gradJO
```
### Gradient step
``` python
def gradient_step(learning_rate, W,b,O):
    lr = learning_rate
    
    # Calcul des gradients
    gradJW, gradJb, gradJO = gradients(Y_pred, Y_true, W, b, O, A)

    # Metez à jour W, b et O 
    # dans la direction oposé du gradient
    # completez
    W -= lr * gradJW
    b -= lr * gradJb
    O -= lr * gradJO

```    
### Boucle d'entrainement
``` python
for i in range(max_iterations):
    #Completez le code ci dessous (numeros 1 a 3)
    # 1) calculez les predictions avec le réseau (forward pass)
    Y_pred, A = MLP(W, b, O)
    
    # 2) Calculez la fonction de cout
    J = cout(Y_pred, y_train)
    
    # Sauvegarder J pour plot
    cost_curve.append(J)
    
    # 3) Calculez les gradients et metez a jour W, b et o avec la fonction gradient_step
    gradient_step(learning_rate, Y_pred, Y_true, W, b, O, A)  
```     
    